In [1]:
import pandas as pd
from pandas import DataFrame as DataFrame
from xlrd import open_workbook
import numpy as np
import re
import xlrd

def df_2_model(db_name,tbl_name,df,sheet):
    cell_name = sheet.cell_value(0,1)
    pattern = re.compile(r".*\((.*?)\).*")

    new_tbl_name = ""
    ss = tbl_name.split("_")
    for s in ss:
        new_tbl_name +=s.capitalize()
    
    model_str= "class {}(models.Model):\n".format(new_tbl_name)
    for r in df.values:
        try:
            if r[0].lower()=="id":
                continue
            type_str = ""
            if r[2].lower().strip()=="int":
                type_str = "models.IntegerField(null=True)"

            elif r[2].lower().strip()=="bigint":   
                type_str = "models.BigIntegerField(null=True)"

            elif r[2].lower().find("datetime")!=-1:   
                type_str = "models.DateTimeField(null=True)"

            elif r[2].lower().find("varchar")!=-1:
                line_list = r[2].lower().split('(')[1][0:-1]
                varchar_len = int(line_list)  
                type_str = "models.CharField(max_length={},null=True)".format(varchar_len)

            elif r[2].lower().strip()=="float":   
                type_str = "models.FloatField(null=True)"            

            elif r[2].lower().find("smallint")!=-1 or r[2].lower().find("tinyint")!=-1:   
                type_str = "models.SmallIntegerField(null=True)"          

            elif r[2].lower().find("text")!=-1:
                type_str = "models.TextField(null=True)"
                
            elif r[2].lower().find("double")!=-1:
                type_str = "models.FloatField(null=True)"

            else:
                print("Error:{}".format(r))

            model_str += "    {} = {}\n".format(r[0].lower(),type_str)
            
        except Exception as e:
            print("Exception:{},{}\n{}".format(new_tbl_name,r,e))

    model_str += "\n    class Meta:\n        db_table = '{}'\n\n".format(tbl_name)
    return model_str

def excel_2_model(db_name,excel_file,sheet_list=[]):
    newexcel = xlrd.open_workbook(excel_file)
    if sheet_list==[]:
        sheet_list=open_workbook(excel_file).sheet_names()
    rettxt = ""
    for sheetname in sheet_list:
        if sheetname.find("版本")==0 or sheetname.find("概述")==0:
            continue
        sheet = newexcel.sheet_by_name(sheetname)
        df = pd.read_excel(excel_file,sheetname,usecols=[1,2,3,4],skiprows=[0,1,2,3,4,5,6])
        rettxt = rettxt + df_2_model(db_name,sheetname,df,sheet)
    return rettxt
        

In [2]:
db_name = "pig"
excel_file = r"D:\mywork\pig\doc\数据字典\天鹏项目_数据字典v1.1.xlsx"
sheet_list = "".split(",")
txt = excel_2_model(db_name,excel_file)
print(txt)

class TblTrackDeviceDetail(models.Model):
    start_time = models.DateTimeField(null=True)
    device_id = models.IntegerField(null=True)
    lng = models.FloatField(null=True)
    lat = models.FloatField(null=True)
    warn_info = models.TextField(null=True)

    class Meta:
        db_table = 'tbl_track_device_detail'

class TblTrackDeviceRtStatus(models.Model):
    start_date = models.DateTimeField(null=True)
    table_name = models.CharField(max_length=50,null=True)

    class Meta:
        db_table = 'tbl_track_device_rt_status'

class TblTrackDeviceRt2(models.Model):
    device_id = models.IntegerField(null=True)
    lng = models.FloatField(null=True)
    lat = models.FloatField(null=True)
    warn_info = models.TextField(null=True)

    class Meta:
        db_table = 'tbl_track_device_rt_2'

class TblTrackDeviceRt1(models.Model):
    device_id = models.IntegerField(null=True)
    lng = models.FloatField(null=True)
    lat = models.FloatField(null=True)
    warn_info = models.Tex